In [183]:
# import libraries test 1
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np                                
import pandas as pd                               
import matplotlib.pyplot as plt                   
from IPython.display import Image                 
from IPython.display import display               
from time import gmtime, strftime                 
from sagemaker.predictor import csv_serializer   
import io
import time
import json
import sagemaker.amazon.common as smac
%matplotlib inline
# Define IAM role
role = get_execution_role()
prefix = 'pred-bank-enroll/sagemaker/DEMO-xgboost-dm'
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'} # each region has its XGBoost container
my_region = boto3.session.Session().region_name # set the region of the instance


print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + containers[my_region] + " container for your SageMaker endpoint.")

Success - the MySageMakerInstance is in the us-east-1 region. You will use the 811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [184]:
role

'arn:aws:iam::079329190341:role/service-role/AmazonSageMaker-ExecutionRole-20190404T141667'

In [187]:
bucket_name = 'demo-saeed'
prefix = 'fraudcredit-local'
dataset_name = 'creditcard.csv'
data_location = 's3://{}/{}/{}'.format(bucket_name, prefix,dataset_name)

In [188]:
data = pd.read_csv(data_location)
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [189]:
from sklearn.preprocessing import StandardScaler
data['normAmount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1, 1))
data['normTime'] = StandardScaler().fit_transform(data['Time'].values.reshape(-1, 1))

data = data.drop(['Time','Amount'],axis=1)
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V22,V23,V24,V25,V26,V27,V28,Class,normAmount,normTime
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0,0.244964,-1.996583
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0,-0.342475,-1.996583
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0,1.160686,-1.996562
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0,0.140534,-1.996562
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0,-0.073403,-1.996541


In [190]:
X = data.loc[:, data.columns != 'Class']
y = data.loc[:, data.columns == 'Class']

In [191]:
X.shape

(284807, 30)

In [192]:
# Number of data points in the minority class
number_records_fraud = len(data[data.Class == 1])
fraud_indices = np.array(data[data.Class == 1].index)

# Picking the indices of the normal classes
normal_indices = data[data.Class == 0].index

# Out of the indices we picked, randomly select "x" number (number_records_fraud)
random_normal_indices = np.random.choice(normal_indices, number_records_fraud, replace = False)
random_normal_indices = np.array(random_normal_indices)

# Appending the 2 indices
under_sample_indices = np.concatenate([fraud_indices,random_normal_indices])

# Under sample dataset
under_sample_data = data.iloc[under_sample_indices,:]

X_undersample = under_sample_data.loc[:, under_sample_data.columns != 'Class']
y_undersample = under_sample_data.loc[:, under_sample_data.columns == 'Class']

# Showing ratio
print("Percentage of normal transactions: ", len(under_sample_data[under_sample_data.Class == 0])/len(under_sample_data))
print("Percentage of fraud transactions: ", len(under_sample_data[under_sample_data.Class == 1])/len(under_sample_data))
print("Total number of transactions in resampled data: ", len(under_sample_data))

Percentage of normal transactions:  0.5
Percentage of fraud transactions:  0.5
Total number of transactions in resampled data:  984


In [193]:
from sklearn.model_selection import train_test_split

# Whole dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 0)

print("Number transactions train dataset: ", len(X_train))
print("Number transactions test dataset: ", len(X_test))
print("Total number of transactions: ", len(X_train)+len(X_test))

# Undersampled dataset
X_train_undersample, X_test_undersample, y_train_undersample, y_test_undersample = train_test_split(X_undersample
                                                                                                   ,y_undersample
                                                                                                   ,test_size = 0.2
                                                                                                   ,random_state = 0)

X_train_undersample, X_val_undersample, y_train_undersample, y_val_undersample = train_test_split(X_train_undersample
                                                                                                   ,y_train_undersample
                                                                                                   ,test_size = 0.2
                                                                                                   ,random_state = 0)
print("")
print("Number transactions train dataset: ", len(X_train_undersample))
print("Number transactions validation dataset: ", len(X_val_undersample))
print("Number transactions test dataset: ", len(X_test_undersample))
print("Total number of transactions: ", len(X_train_undersample)+len(X_val_undersample)+len(X_test_undersample))

Number transactions train dataset:  227845
Number transactions test dataset:  56962
Total number of transactions:  284807

Number transactions train dataset:  629
Number transactions validation dataset:  158
Number transactions test dataset:  197
Total number of transactions:  984


### Upload training set

In [194]:
f = io.BytesIO()
smac.write_numpy_to_dense_tensor(f, X_train_undersample.values.astype('float32'), y_train_undersample.values.reshape(-1).astype('float32'))
f.seek(0)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train','linear_train.data')).upload_fileobj(f)
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket_name, prefix))
s3_input_train.config

{'DataSource': {'S3DataSource': {'S3DataDistributionType': 'FullyReplicated',
   'S3DataType': 'S3Prefix',
   'S3Uri': 's3://demo-saeed/fraudcredit-local/train'}}}

### Upload validation set

In [195]:
f = io.BytesIO()
smac.write_numpy_to_dense_tensor(f, X_val_undersample.values.astype('float32'), y_val_undersample.values.reshape(-1).astype('float32'))
f.seek(0)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'validation','linear_val.data')).upload_fileobj(f)
s3_input_validation = sagemaker.s3_input(s3_data='s3://{}/{}/validation'.format(bucket_name, prefix))
s3_input_validation.config


{'DataSource': {'S3DataSource': {'S3DataDistributionType': 'FullyReplicated',
   'S3DataType': 'S3Prefix',
   'S3Uri': 's3://demo-saeed/fraudcredit-local/validation'}}}

### Upload test set


In [196]:
test_file = 'linear_test.data'

f = io.BytesIO()
smac.write_numpy_to_dense_tensor(f, X_test_undersample.values.astype('float32'), y_test_undersample.values.reshape(-1).astype('float32'))
f.seek(0)

boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test',test_file)).upload_fileobj(f)
s3_input_test = sagemaker.s3_input(s3_data='s3://{}/{}/test'.format(bucket_name, prefix))
s3_input_test.config

{'DataSource': {'S3DataSource': {'S3DataDistributionType': 'FullyReplicated',
   'S3DataType': 'S3Prefix',
   'S3Uri': 's3://demo-saeed/fraudcredit-local/test'}}}

# training

In [197]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'linear-learner')
container

'382416733822.dkr.ecr.us-east-1.amazonaws.com/linear-learner:1'

In [198]:
output_location = 's3://{}/{}/output'.format(bucket_name, prefix)
print('training artifacts will be uploaded to: {}'.format(output_location))

training artifacts will be uploaded to: s3://demo-saeed/fraudcredit-local/output


In [199]:
X_train_undersample.shape

(629, 30)

In [200]:
import boto3
import sagemaker

sess = sagemaker.Session()

linear = sagemaker.estimator.Estimator(container,
                                       role, 
                                       train_instance_count=1, 
                                       train_instance_type='ml.c4.xlarge',
                                       output_path=output_location,
                                       sagemaker_session=sess)
linear.set_hyperparameters(feature_dim=30,
                           predictor_type='binary_classifier',
                           mini_batch_size=20)

linear.fit({'train': s3_input_train, 'test': s3_input_test, 'validation': s3_input_validation})

2019-05-22 04:39:43 Starting - Starting the training job...
2019-05-22 04:39:45 Starting - Launching requested ML instances.........
2019-05-22 04:41:26 Starting - Preparing the instances for training.........
2019-05-22 04:43:06 Downloading - Downloading input data
2019-05-22 04:43:06 Training - Downloading the training image.
Docker entrypoint called with argument(s): train
[05/22/2019 04:43:11 INFO 140486093657920] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5', u'bias_lr_mult': u'auto', u'lr_scheduler_step': u'auto', u'init_method': u'uniform', u'init_sigma': u'0.01', u'lr_scheduler_minimum_lr': u'auto', u'target_recall': u'0.8'

In [201]:
linear.output_path

's3://demo-saeed/fraudcredit-local/output'

In [203]:
#linear.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')
linear_predictor = linear.deploy(initial_instance_count=1, instance_type='ml.t2.medium', endpoint_name='aws-linear-learner', update_endpoint=True)




In [207]:
from sagemaker.predictor import csv_serializer, json_deserializer

linear_predictor.content_type = 'text/csv'
linear_predictor.serializer = csv_serializer
linear_predictor.deserializer = json_deserializer
linear_predictor.predict(X_test.iloc[0])

{'predictions': [{'score': 0.11748139560222626, 'predicted_label': 0.0}]}

In [208]:
linear_predictor.endpoint

'aws-linear-learner'

In [ ]:
input_Data = ",".join( map( str, X_test.iloc[0] ) )
input_Data

In [209]:
import io
import boto3
import json
import csv

# grab environment variables
ENDPOINT_NAME = 'aws-linear-learner'
runtime= boto3.client('runtime.sagemaker')

input_Data = ",".join( map( str, X_test.iloc[0] ) )
response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                   ContentType='text/csv',
                                   Body=input_Data)
print(response)
result = json.loads(response['Body'].read().decode())
print(result)


{'ResponseMetadata': {'RequestId': 'c4f995d6-db76-4473-bbb3-02254f7a4491', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'c4f995d6-db76-4473-bbb3-02254f7a4491', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Wed, 22 May 2019 13:47:03 GMT', 'content-type': 'application/json', 'content-length': '73'}, 'RetryAttempts': 0}, 'ContentType': 'application/json', 'InvokedProductionVariant': 'AllTraffic', 'Body': <botocore.response.StreamingBody object at 0x7ff5d6227908>}
{'predictions': [{'score': 0.11748139560222626, 'predicted_label': 0.0}]}
